In [2]:
import scipy.io
import numpy as np


from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

In [46]:
# Load the .mat file
mat_file = scipy.io.loadmat('/Users/chrisdollo/Documents/coding_projects/EMG/Matlab/gestureTable_clean.mat')
cell_array = mat_file['finalCellArray']

X = []
Y = []

for gesture_type in range(cell_array.shape[1]):
    for row_idx in range(cell_array.shape[0]):
        cell = cell_array[row_idx, gesture_type]

        # Convert to numpy array explicitly and ensure shape (24, 18)
        cell = np.array(cell)
        if cell.shape != (24, 18):
            raise ValueError(f"Unexpected shape: {cell.shape} at row {row_idx}, gesture {gesture_type}")

        # Add a new axis for channel (CNN expects 4D input: samples, height, width, channels)
        X.append(cell.astype(np.float32))
        Y.append(gesture_type)

# Convert list to NumPy arrays
X = np.array(X, dtype=np.float32)  # shape: (samples, 24, 18)
X = X[..., np.newaxis]             # shape: (samples, 24, 18, 1)
Y = np.array(Y, dtype=np.int32)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.2, stratify=Y, random_state=42
)

print("X shape:", X.shape)
print("y shape:", Y.shape)


✅ X shape: (588, 24, 18, 1)
✅ y shape: (588,)


In [47]:
X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.2, random_state=42, stratify=Y, shuffle=True
)

print(f"Training samples: {X_train.shape[0]}")
print(f"Testing samples: {X_test.shape[0]}")

Training samples: 470
Testing samples: 118


Let's build the model.

For this task we will be using a Convolutional Neural Network

In [48]:
def buil_model(input_shape=(24, 18, 1), num_classes=7):

    model = Sequential()

    # Conv Layer 1
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))

    # Conv Layer 2
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))

    # Flatten and Dense Layers
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(num_classes, activation='softmax'))

    # Compile
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

In [49]:
model = buil_model()

/Users/chrisdollo/anaconda3/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [51]:
# Train the model
history = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=16,
    validation_split=0.1,  # Use 10% of training set for validation
    verbose=1
)

Epoch 1/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.1275 - loss: 1.9422 - val_accuracy: 0.1702 - val_loss: 1.9558
Epoch 2/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.1657 - loss: 1.9453 - val_accuracy: 0.1489 - val_loss: 1.9551
Epoch 3/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.1783 - loss: 1.9466 - val_accuracy: 0.0851 - val_loss: 1.9561
Epoch 4/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.1290 - loss: 1.9469 - val_accuracy: 0.0851 - val_loss: 1.9563
Epoch 5/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.1756 - loss: 1.9404 - val_accuracy: 0.0851 - val_loss: 1.9548
Epoch 6/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.1500 - loss: 1.9458 - val_accuracy: 0.0851 - val_loss: 1.9523
Epoch 7/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.1543 - loss: 1.9402 - val_accuracy: 0.0851 - val_loss: 1.9593
Epoch 8/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.1303 - loss: 1.9481 - val_accuracy: 0.1064 - val_loss:

In [52]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=1)

print(f"✅ Test Accuracy: {test_accuracy * 100:.2f}%")
print(f"📉 Test Loss: {test_loss:.4f}")


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0688 - loss: 1.9609 
✅ Test Accuracy: 10.17%
📉 Test Loss: 1.9585


In [53]:
# After training
final_train_acc = history.history['accuracy'][-1]
final_train_loss = history.history['loss'][-1]

print(f"✅ Final Training Accuracy: {final_train_acc * 100:.2f}%")
print(f"📉 Final Training Loss: {final_train_loss:.4f}")


✅ Final Training Accuracy: 15.37%
📉 Final Training Loss: 1.9455
